In [1]:
import re
import json
from lxml import html
import urllib.request

In [2]:
URL_BASE = 'http://www.mathnet.ru'

URL = '''
    http://www.mathnet.ru/php/
    archive.phtml?jrnid=uzku&wshow=issue&bshow=contents&series=0&year=2017
    &volume=158&issue=1&option_lang=rus&bookID=1621
'''

In [3]:
ANNOTATION_XPATH_EXPR = '''
    //b[contains(text(), 'Аннотация')]/following-sibling::text()[not(preceding-sibling::b[contains(text(), 'Ключевые')])]
'''

TITLE_XPATH_EXPR = '''
    //td//td//td//a[@class='SLink']
'''

KEYWORDS_XPATH_EXPR = '''
    //b[contains(text(), 'Ключевые')]/following-sibling::i
'''

In [4]:
def main():
    main_page = html.fromstring(urllib.request.urlopen(URL).read())

    articles = []
    
    links = list(filter(lambda x: x.text is not None, main_page.xpath(TITLE_XPATH_EXPR)))
    for link in links:
        article_page = html.fromstring(urllib.request.urlopen(URL_BASE + link.get("href")).read())

        annotation = str.join('', article_page.xpath(ANNOTATION_XPATH_EXPR))
        annotation = re.sub('[^a-zA-ZА-Яа-я0-9]+', ' ', annotation)
        
        title = link.text;
        
        keywords = re.split(', ', article_page.xpath(KEYWORDS_XPATH_EXPR)[0].text)

        articles.append({
            'title': title,
            'link': link.get('href'),
            'annotation': annotation,
            'keywords': keywords
        })

    result = {
        'issue': {
            'URL': URL,
            'articles': articles
        }
    }
        
    print(json.dumps(result, ensure_ascii=False, indent=4, sort_keys=True))

In [5]:
main()

{
    "issue": {
        "URL": "http://www.mathnet.ru/php/archive.phtml?jrnid=uzku&wshow=issue&bshow=contents&series=0&year=2017&volume=158&issue=1&option_lang=rus&bookID=1621",
        "articles": [
            {
                "annotation": " В работе рассмотрен класс многочленов типа Капелли в свободной ассоциативной алгебре где произвольное поле счетное множество обобщающий конструкцию кратных многочленов Капелли Приведены основные свойства введенных многочленов В частности указано их разложение через многочлены того же вида и установлены некоторые соотношения между их идеалами Кроме того установлена связь между двойными многочленами Капелли и квазимногочленами Капелли ",
                "keywords": [
                    "матричная алгебра",
                    "многочлен Капелли",
                    "полиномиальное тождество",
                    "свободная ассоциативная алгебра",
                    "симметрическая группа",
                    "стандартный многочлен",
        